In [49]:

#GETTING MY ID:
import json

id = open('id.json', 'r')
mi = json.load(id)
azure_id = mi['azure_id']

from azureml.core import Workspace

ws = Workspace.create(name="Bankruptancy_test",
                      subscription_id = azure_id,
                      resource_group = "modelo",
                      location = "centralindia")

Deploying AppInsights with name bankruptinsights345e99cf.
Deployed AppInsights with name bankruptinsights345e99cf. Took 6.61 seconds.
Deploying KeyVault with name bankruptkeyvault6d7f4f36.
Deploying StorageAccount with name bankruptstorage956a13d0c.
Deployed KeyVault with name bankruptkeyvault6d7f4f36. Took 23.9 seconds.
Deploying Workspace with name Bankruptancy_test.
Deployed StorageAccount with name bankruptstorage956a13d0c. Took 38.19 seconds.
Deployed Workspace with name Bankruptancy_test. Took 42.98 seconds.


In [50]:
from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [51]:
################ score.py ###################

scorepy = """
import json
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    # Replace 'model.pkl' with the actual name of your registered model file if different
    model_path = Model.get_model_path('model')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        # Load the JSON input and convert to a DataFrame
        input_data = json.loads(raw_data)
        data = pd.DataFrame(input_data['data'])
        
        # Define the expected feature columns (ensure these match the training features)
        feature_columns = [
            ' ROA(C) before interest and depreciation before interest',
            ' ROA(A) before interest and % after tax',
            ' ROA(B) before interest and depreciation after tax',
            ' Net Value Per Share (B)',
            ' Net Value Per Share (A)',
            ' Net Value Per Share (C)',
            ' Persistent EPS in the Last Four Seasons',
            ' Operating Profit Per Share (Yuan ¥)',
            ' Per Share Net profit before tax (Yuan ¥)',
            ' Debt ratio %',
            ' Net worth/Assets',
            ' Borrowing dependency',
            ' Operating profit/Paid-in capital',
            ' Inventory and accounts receivable/Net value',
            ' Working Capital to Total Assets',
            ' Current Liability to Assets',
            ' Working Capital/Equity',
            ' Current Liabilities/Equity',
            ' Retained Earnings to Total Assets',
            ' Total expense/Assets',
            ' Current Liability to Equity',
            ' Equity to Long-term Liability',
            ' Current Liability to Current Assets',
            ' Liability-Assets Flag',
            ' Net Income to Total Assets',
            " Net Income to Stockholder's Equity",
            ' Liability to Equity'
        ]
        
        # Select only the features needed for prediction
        data_features = data[feature_columns]
        
        # Make predictions with the loaded model
        result = model.predict(data_features).tolist()
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"error": str(e)})
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [52]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn','numpy','joblib']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/var/folders/b6/wkc18d0n5vn3fsnd8xjknlch0000gp/T/ipykernel_31642/1741792357.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [53]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
